In [1]:
:opt no-lint
{-# LANGUAGE ScopedTypeVariables TypeApplications
             RankNTypes          KindSignatures   #-}

## 계산이 끝나지 않는 람다식
람다식 작은걸음 $(\lambda x.x\;x)\;e_2 \longmapsto e_2\;e_2$를 생각해 보자.
여기서 $e_2$가 적용하는 함수 $(\lambda x.x\;x)$와 똑같은 식인 경우에 어떤 일이 벌어질까?
바로 아래와 같이 된다.
\newline\indent
$(\lambda x.x\;x)(\lambda x.x\;x) \longmapsto (\lambda x.x\;x)(\lambda x.x\;x) \longmapsto (\lambda x.x\;x)(\lambda x.x\;x) \longmapsto \ldots$
\newline
$\beta$규칙에 따라 함수 몸체 $x\;x$에서 $x$를 $(\lambda x.x\;x)$로 바꿔치면
원래 처음 시작했던 식인 $(\lambda x.x\;x)(\lambda x.x\;x)$가 되어버린다.
그래서 아무리 여러 번 $\beta$줄임을 반복해도 제자리걸음만 할 뿐 계산이 끝나지 않는다.
이제 드브루인 인덱스(그림 \ref{fig:ChurchDeBruijn})를 활용해 타입없는 람다계산법에서 끝나지
않는 계산을 하스켈로 다뤄보자. 우선 람다계산법의 문법구조를 아래와 같은
데이터 타입으로 선언한다.

In [2]:
data DExpr = DI Int | DAbs DExpr | DApp DExpr DExpr deriving (Eq,Ord)

In [3]:
-- GHC가 자동으로 deriving하는 Show 인스턴스가 아닌  더 간략한 출력을 위해
instance Show DExpr where   -- 직접 작성한 DExpr에 대한 Show 인스턴스
  showsPrec _ (DI i) = showString (show i)
  showsPrec p (DAbs e) = showParen (p > 1) $
              showString "\\" . showsPrec 1 e
  showsPrec p (DApp e1 e2) = showParen (p > 9) $
              showsPrec 9 e1 . showString " " . showsPrec 10 e2

In [4]:
DAbs (DAbs (DI 1 `DApp` DI 2))        -- \x.\y.x y
DAbs (DI 1) `DApp` DAbs (DI 1)        -- (\x.x)(\x.x)
DAbs ((DI 1 `DApp` DI 1) `DApp` DI 1) -- \x.x x x
DAbs (DI 1 `DApp` (DI 1 `DApp` DI 1)) -- \x.x (x x)

\\1 2

(\1) (\1)

\1 1 1

\1 (1 1)

\noindent
그리고 매개변수에 넘겨받은 인자를
\index{자기 자신에 적용|see{self-application}}\index{self-application|see{자기 자신에 적용}}자기 자신에 적용(self-appliation)하는 함수 $\lambda x.x\;x$는
드브루인 인덱스를 활용한 표현으로는 $\lambda 1~1$이다. 이를 `w`로 선언해 놓고 앞으로 의미구조를
하스켈로 구현하여 `w w`가 제자리걸음함을 확인해 보자.

In [5]:
w = DAbs (DI 1 `DApp` DI 1) -- \x.x x

w         -- self-application 함수인 w
DApp w w  -- w를 자기 자신에 적용한 식

\1 1

(\1 1) (\1 1)

작은걸음 의미구조는 $\beta$규칙 및 맥락 규칙들로 정의할 수 있다.
\newline\vspace*{-2ex}
$$(\lambda\,e)\;e_2 \longmapsto \;\downarrow_0^1 \{1{\mapsto}(\uparrow_0^1 e_2)\}e$$
$$\frac{~ e_1\longmapsto e_1'~}{~ e_1~e_2\longmapsto e_1'~e_2 ~}
\qquad\frac{~ e_2\longmapsto e_2'~}{~ e_1~e_2\longmapsto e_1~e_2' ~}
\qquad\frac{~ e\longmapsto e' ~}{~ \lambda\,e \longmapsto \lambda\,e' ~}
$$
위에서 $\downarrow_m^k$, $\uparrow_m^k$와 바꿔치기 $\{i{\mapsto}e\}$가 어떻게 동작하는지만 확실히 하면
나머지 맥락 규칙에 대한 처리는 산술식 작은걸음 계산기인 `step`을 작성하듯 선언하면 된다.
$\downarrow_m^k = \uparrow_m^{-k}$이며, 일반적으로 $\downarrow_m^k e$는 식 $e$에서
$m$보다 큰 드브루인 인덱스를 $k$만큼씩 증가시킨 식을 나타내는 표현으로 왼쪽 아래처럼 정의할 수 있다.
함수요약식에 대한 정의에서 함수 전체에는 $m$이다가 함수 몸체에서 $m+1$이
되는 이유는 바인더 $\lambda$가 한 겹 더 감싸진 몸체 안에서는 그 바깥에서와
같은 대상을 나타내는 드브루인 인덱스가 1씩  더 큰 수가 되기 때문이다.
바꿔치기 $\{i{\mapsto}e\}$는 $\uparrow_m^k$ 표현을 활용해 오른쪽 아래와 같이 정의할 수 있다.
$$\begin{array}{ll}
\uparrow_m^k i 
\!\!\!\!&= \begin{cases}i+k & (i>m) \\ i & (i\le m)\end{cases} \\
\uparrow_m^k (\lambda\,e)
\!\!\!\!&=~ \lambda\,(\uparrow_{m{+}1}^k e) \\
\uparrow_m^k (e_1\;e_2)
\!\!\!\!&=~ (\uparrow_m^k e_1)\;(\uparrow_m^k e_2)
\end{array}
\qquad~~
\begin{array}{ll}
\{i{\mapsto}e\}j 
\!\!\!\!&= \begin{cases}e & (i=j) \\ j & (i\neq j)\end{cases}\\
\{i{\mapsto}e\}(\lambda\, e_1) 
\!\!\!\!&=~ \lambda\, \left\{(i{+}1){\mapsto}(\uparrow_0^1 e)\right\}e_1 \\
\{i{\mapsto}e\}(e_1\;e_2)
\!\!\!\!&=~ \{i{\mapsto}e\}e_1~\{i{\mapsto}e\}e_2 \\
\end{array}$$
위 정의를 하스켈로 옮기면 다음과 같다.

In [6]:
up m k (DI i)       = DI (if i>m then i+k else i)
up m k (DAbs e)     = DAbs (up (m+1) k e)
up m k (DApp e1 e2) = DApp (up m k e1) (up m k e2)

dn m k = up m (-k)

subst i e (DI j)       = if i==j then e else DI j
subst i e (DAbs e1)    = DAbs (subst (i+1) (up 0 1 e) e1) 
subst i e (DApp e1 e2) = DApp (subst i e e1) (subst i e e2)

In [7]:
:type up
:type dn
:type subst

up :: Int -> Int -> DExpr -> DExpr

dn :: Int -> Int -> DExpr -> DExpr

subst :: Int -> DExpr -> DExpr -> DExpr

\noindent
위의 `up`, `dn`, `subst`를 활용해 다음과 같이 하스켈로 $\beta$규칙을 작성할 수 있다.

In [8]:
beta :: DExpr -> [DExpr]  -- 일단 베타 규칙만 작성
beta (DApp (DAbs e) e2) = [ dn 0 1 (subst 1 (up 0 1 e2) e) ]
beta _                  = []

In [9]:
DApp w w        -- (\x.x) (\x.x)
beta (DApp w w) -- 제자리걸음 확인!

(\1 1) (\1 1)

[(\1 1) (\1 1)]

In [10]:
step :: DExpr -> [DExpr]
step (DI _)       = []
step (DApp e1 e2) = undefined -- undefined 대신 의미규칙대로
step (DAbs e)     = undefined -- step 함수를 완성하는 연습문제

In [11]:
step :: DExpr -> [DExpr]
step (DI _)       = []
step (DApp e1 e2) = beta (DApp e1 e2)
                 ++ [DApp e1' e2 | e1' <- step e1]
                 ++ [DApp e1 e2' | e2' <- step e2]
step (DAbs e)     = [DAbs e' | e' <- step e]

In [12]:
-- step이 제대로 동작하는지 다음 내용으로 테스트해 보라
eee1 = DAbs . DAbs $ DI 1 `DApp` DI 2 `DApp` DI 3
eee2 = DAbs . DAbs $ DI 1 `DApp` DI 3
DAbs $ DApp eee1 eee2 -- \x.(\y.\z.z y x) (\w.\v.v x)
step it               -- \x.\y.\z.z (\w.\v.v x) x

\(\\1 2 3) (\\1 3)

[\\1 (\\1 4) 2]

## 타입없는 람다계산법에서 고정점(fixpoint)
\label{sec:fixpoint}
수학에서 함수 $f$의 고정점(fixpoint)이란 $x \equiv f(x)$인 $x$를 말한다.
\index{타입없는 람다계산법}\index{untyped $\lambda$-calculus}타입없는 람다계산법에는 아무 함수에 적용해 그 함수의 고정점을 계산하는
\index{고정점 조합자|see{fixpoint combinator}}\index{fixpoint combinator|see{고정점 조합자}}고정점 조합자(fixpoint combinator)가 여럿 존재한다.
하스켈 커리가 발견한 $Y=\lambda f.(\lambda x.f\,(x\;x))\,(\lambda x.f\,(x\;x))$는
\index{Y-combinator|see{Y조합자}}\index{Y조합자|see{Y-combinator}}
Y조합자(Y-combinator)라 불리는 람다계산법의 대표적인 고정점 조합자로,
다음과 같이 $Y\,f \equiv_\beta f(Y\,f)$를 만족함을 보일 수 있다.\vspace*{-2ex}
\begin{align*}
Y\,f \longmapsto (\lambda x.f\,(x\;x))\,(\lambda x.f\,(x\;x)) \longmapsto f\,((\lambda x.f\,(x\;x))\,(\lambda x.f\,(x\;x))) &\\
f(Y\,f) \longmapsto f\,((\lambda x.f\,(x\;x))\,(\lambda x.f\,(x\;x))) &
\end{align*}
이런 고정점의 성질을 활용하면
$Y\,f \equiv_\beta f(Y\,f) \equiv_\beta f(f(Y\,f)) \equiv_\beta \cdots$처럼
함수 $f$를 몇 번이고 반복해서 적용하는 계산을 표현할 수 있다.

Y조합자를 드브루인 인덱스를 활용하는 람다식을 나타내는 하스켈 데이터 타입인 `DExpr`로
표현하여 다음과 같이 `yC`로 선언하자.

In [13]:
yC = DAbs (DApp e211 e211) -- \f.(\x.f(x x))(\x.f(x x))
   where e211 = DAbs (DApp (DI 2) (DI 1 `DApp` DI 1))

In [14]:
yC -- 드브루인 인덱스로 나타낸 Y조합자

\(\2 (1 1)) (\2 (1 1))

\newpage
Y조합자를 항등함수($\lambda z.z$)에 적용하면 세 작은걸음만에 앞서 살펴본 계산이 끝나지 않는 람다식을 유도할 수 있다. 
\vspace*{-2ex}
\begin{align*}
(\lambda f.(\lambda x.f\,(x\;x))\,(\lambda x.f\,(x\;x)))~(\lambda z.z)
\longmapsto\;& (\lambda x.(\lambda z.z)(x\;x))\,(\lambda x.(\lambda z.z)(x\;x)) \\
\longmapsto\;& (\lambda x.x\;x)\,(\lambda x.(\lambda z.z)(x\;x)) \\
\longmapsto\;& (\lambda x.x\;x)\,(\lambda x.x\;x)
\end{align*}
산술식에서처럼 `step`을 확장한 `step'`를 고차함수 `hat`을 통해 선언하자.

In [15]:
import Data.List (nub)
hat f es = nub (concat [f e| e <- es])

step' :: [DExpr] -> [DExpr]
step' = hat step

In [16]:
DApp yC (DAbs (DI 1)) -- yC (\x.x)

(\(\2 (1 1)) (\2 (1 1))) (\1)

`step'` 함수를 세 번 연달아 적용한 결과로 나오는 리스트의 모든 원소를
다음과 같이 한 줄에 하나씩 출력해 보자. 앞서 `step` 함수를 제대로 완성했다면,
아래의 출력 결과 중에 계산이 끝나지 않는 람다식
$(\lambda x.x~x)~(\lambda x.x~x)$에 해당하는 `(\1 1) (\1 1)`도
출력됨을 확인할 수 있을 것이다. 참고로 리스트의 원소들이 출력되는
순서는 `step`을 작성한 방식에 달라질 수도 있다.

In [17]:
-- yC (\x.x)로부터 세 작은걸음 진행한 모든 람다식을 한 줄에 하나씩 출력
mapM_ print $ (step' . step' . step') [ DApp yC (DAbs (DI 1)) ]

(\(\1) (1 1)) (\(\1) (1 1))
(\1) ((\1) ((\(\1) (1 1)) (\(\1) (1 1))))
(\1) ((\1 1) (\(\1) (1 1)))
(\1) ((\(\1) (1 1)) (\1 1))
(\1 1) (\1 1)
(\1) ((\1 1) (\1 1))
(\1 (1 (1 ((\2 (1 1)) (\2 (1 1)))))) (\1)

## 하스켈에서 고정점 조합자 선언하기
\label{sec:fixHaskell}
타입없는 람다계산법에서 자기 자신에 대한 적용(self-application)을
이용해 끝나지 않는 계산을 표현하거나 Y조합자와 같은 고정점 조합자를
람다식으로 정의할 수 있음을 알아보았다. 그렇지만 하스켈에서는 순수한
람다식만으로 끝나지 않는 계산이나 고정점 조합자를 표현하기는 어렵다.
왜냐하면 하스켈은 타입있는 람다계산법을 기반으로 하는데,
타입있는 람다계산법에서는 자기 자신에 대한 적용 $x~x$에
적절한 타입을 부여하기 어렵기 때문이다. 인자 부분인 오른쪽의
$x:\tau$라면 함수 부분인 왼쪽의 $x:\tau\to\tau_1$여야 한다.
단순타입 람다계산법(STLC)에서는 같은 식의 타입이
$\tau$이면서 $\tau\to\tau_1$이기도 하기란 불가능하다. 앞으로 소개할
다형 람다계산법에서는 자기 자신에 대한 적용에 타입을 부여할 수는 있지만
무한한 계산을 표현할 수는 없다. 애초에 타입있는 람다계산법을 고안하게 된 동기가
바로 끝없는 계산처럼 논리적 모순에 대응되는 람다식을 허용하지 않기 위해서다.
그래서 튜링완전한 범용 함수형 언어에서는 람다식과 별도로
재귀적 표현을 위한 문법요소를 제공한다.

In [18]:
:type \x -> x x  -- 하스켈에서 self-application 람다식은 타입 오류

: 

그런데 만일 하스켈에서 딱 한 번만 재귀함수를 선언할 수 있고 다른 곳에서는
재귀적인 표현을 허용하지 않도록 제약을 가한다면 우리는 어떤 재귀함수를 선언해야 할까?
다음과 같이 고정점 조합자를 딱 한 번만 선언하면 다른 곳에서 재귀적 표현이 금지되더라도
일반적인 재귀함수를 얼마든지 만들어낼 수 있다.

In [19]:
fix f = f (fix f) -- 고정점 조합자의 성질을 재귀함수로 선언

In [20]:
:type fix

fix :: forall t. (t -> t) -> t

정수 $n$을 받아 팩토리얼 $n!$를 계산하는 함수를 두 가지 방법으로 선언하여 비교해 보자.
처음에는 직접 재귀함수로 선언해 보고, 그 다음에는
고정점 조합자 `fix`와 다른 재귀적이지 않은 함수를 활용해 선언해 보겠다.
프로그램의 단순화를 위해 팩토리얼 함수를 음수에 적용하는 것은 고려하지 않고
0이상의 자연수에만 적용한다고 가정하자.

In [21]:
fact :: Integer -> Integer -- 직접 재귀함수로 팩토리얼 선언
fact 0 = 1                 -- 0! = 1
fact n = n * fact (n-1)    -- n! = n * (n-1)!

In [22]:
fact 0
fact 5

1

120

In [23]:
factF :: (Integer -> Integer) -> (Integer -> Integer)
factF f 0 = 1           -- f를 원래 파라미터 앞에 추가
factF f n = n * f (n-1) -- 재귀함수 호출 위치에 f 사용

fact' = fix factF -- 고정점 조합자를 활용한 팩토리얼 선언

In [24]:
:type fact'

fact' :: Integer -> Integer

In [25]:
fact' 0
fact' 5

1

120

# 다형 람다계산법
\label{sec:SystemF}

문법구조\vspace{-4.5ex}
\begin{align*}
\qquad
t \in B & ~\text{\small(기본타입)} &
\tau\in T ~::= ~& t ~\mid~ \tau \to \tau
~\mid~ \alpha ~\mid~ \forall \alpha.\tau
\\
c \in C & ~\text{\small(기본상수)} &
e\in E ~::=~ & c \mid x \mid \lambda x{\,:\,}\tau.\,e \mid e~e
\mid \Lambda \alpha.\,e \mid e\,@\tau
\end{align*} 타입규칙
$\qquad
\Delta ~::=~ c{\,:\,}\tau,\Delta \mid \bm{\cdot} \qquad\qquad\qquad
\Gamma ~::=~ \alpha{\,:\,}\star,\Gamma \mid x{\,:\,}\tau,\Gamma \mid \bm{\cdot}$
$$
{\scriptstyle(\textsc{Con})}
\frac{~ ~}{~ \Gamma \vdash c:\tau ~}
({\scriptstyle c{\,:\,}\tau\;\in\,\Delta})
\qquad\qquad
{\scriptstyle(\textsc{Var})}
\frac{~ ~}{~ \Gamma \vdash x:\tau ~}
({\scriptstyle x{\,:\,}\tau\;\in\,\Gamma})
$$ \vspace*{-.5ex}
$$
{\scriptstyle(\textsc{Abs})}
\frac{~ x:\tau_2,\Gamma \vdash e:\tau ~}{~ \Gamma \vdash \lambda x{\,:\,}\tau_2.\,e : \tau_2\to\tau ~}
\qquad
{\scriptstyle(\textsc{App})}
\frac{~ \Gamma \vdash e_1:\tau_2\to\tau ~\quad~ \Gamma \vdash e_2:\tau_2 ~}{~ \Gamma \vdash e_1\;e_2 : \tau ~}
$$ \vspace*{-.5ex}
$$
{\scriptstyle(\textsc{TAbs})}
\frac{~ \alpha{\,:\,}\star,\Gamma \vdash e:\tau ~}{~ \Gamma \vdash \Lambda \alpha.\,e : \forall \alpha.\tau ~}
\qquad
{\scriptstyle(\textsc{TApp})}
\frac{~ \Gamma \vdash e:\forall\alpha.\tau ~}{~ \Gamma \vdash e\,@\tau_1 : \{\alpha{\mapsto}\tau_1\}\tau ~}
$$
$~$\newline
의미구조
$\displaystyle
 \quad {\scriptstyle(\beta_\to)}\frac{~~}{~(\lambda x{:}\tau.\,e)\,e_2 \longmapsto \{x{\mapsto}e_2\}e~}
\qquad {\scriptstyle(\beta_\forall)}\frac{~~}{~(\Lambda \alpha.\,e)\,@\tau \longmapsto \{\alpha{\mapsto}\tau\}e~}$
$$ \mathcal{E} ~::=~ \bullet
\mid \lambda x{\,:\,}\tau.\,\mathcal{E} 
\mid \mathcal{E}~e \mid e~\mathcal{E}
\mid \Lambda \alpha.\,\mathcal{E} \mid \mathcal{E}\,@\tau
\qquad
{\scriptstyle(\mathcal{E})}\frac{~e\longmapsto e'~}{~\mathcal{E}[e]\xmapsto{~_C~}\mathcal{E}[e']~}
$$

\index{람다계산법!타입있는 람다계산법!다형 람다계산법}\index{$\lambda$-calculus!typed $\lambda$-calculus!polymorphic $\lambda$-calculus}프랑스의 논리학자인 장-이브 지라르\cite{Girard1971}는
\index{System F|see{다형 람다계산법}}\index{System F|see{polymorphic $\lambda$-calculus}}System F라는 이름의,
미국의 컴퓨터 과학자 존 레이놀즈\cite{Reynolds1974PolyLambda}는
이후
\index{다형 람다계산법|see{polymorphic $\lambda$-calculus}}\index{polymorphic $\lambda$-calculus|see{다형 람다계산법}}다형 람다계산법(polymorphic $\lambda$-calculus) 혹은
다형타입 람다계산법(polymoprphically typed $\lambda$-calculus)으로 불리게 된,
알고보니 똑같은 타입있는 람다계산법(그림 \ref{fig:SystemF})을 1970년대 초반에 각자 독립적으로 고안했다.
\`다형 람다계산법'보다는 \`System F'가 더 짧으므로 주로 System F로 부르겠다.
System F의 타입은 STLC의 타입을 구성하는 요소 외에도 타입변수($\alpha$)와 다형타입($\forall\alpha.\tau$)을 포함한다.
System F의 식은 STLC의 식을 구성하는 요소 외에도 타입요약식(type abstraction)과 타입적용식(type application)을 포함한다. 함수요약식($\lambda x{:}\tau.e$)이 식을 식에 대응시키는 함수를 표현한다면
타입요약식($\Lambda\alpha.e$)은 타입을 식에 대응시킨키는 함수를 표현한다.
식을 식에 적용하는 함수적용식($e_1~e_2$)과의 차이를 뚜렷이 드러내기 위해
식을 타입에 적용하는 타입적용식($e\;@\tau$)에서는 인자로 넘기는 타입 앞에 $@$기호를 표기한다.
참고로, 표준적인 타입적용식의 표기는 레이놀즈의 논문\cite{Reynolds1974PolyLambda}에서처럼
$e[\tau]$이다. 하지만 실행맥락의 구멍을 채워넣는 표현($\mathcal{E}[e]$)에도 같은 괄호를 사용하므로,
여기서는 혼동의 여지를 줄이고자, 하스켈에서 명시적인 타입 적용을 표현하도록 지원하는
GHC확장 기능의 문법을 따라 $e\;@\tau$와 같이 표기하였다.

타입요약식에 대한 타입규칙($\scriptstyle\textsc{TAbs}$)은 타입요약식이 다형타입이 됨을 규정한다.
타입적용식에 대한 타입규칙($\scriptstyle\textsc{TApp}$)은
왼쪽의 적용하는 식($e$)이 다형타입($\forall\alpha.\tau$)이어야 하며
전체 타입적용식($e\,@\tau_1$)의 타입은 $\tau$에 자유로이 나타나는
$\alpha$를 오른쪽의 타입 인자 $\tau_1$로 구체화한 타입이 됨을 규정한다.
그 외의 다른 타입규칙은 STLC에서와 같다.

그림 \ref{fig:SystemF}의 의미구조는 $\beta_\to$규칙과 $\beta_\forall$규칙을
적용하는 순서를 제약하지 않는 단순한 형태의 의미구조이다. System F를 바탕으로 하는
정적 타입 함수형 언어의 실제 구현은 실행 전의 타입 검사 단계에서
$\beta_\forall$규칙만을 미리 적용해 타입을 가능한 한 구체화해 놓고,
실행 시간에 $\beta_\to$규칙을 적용하며 계산을 진행하는 것으로 이해할 수 있다. 

## 다형 람다계산법과 하스켈

In [26]:
:type id  -- 표준 라이브러리 항등함수 id는 다형타입

id :: forall a. a -> a

In [27]:
:type id @Int     -- Int에 명시적 타입 적용
:type id @String  -- String에 명시적 타입 적용

id @Int :: Int -> Int

id @String :: String -> String

In [28]:
id      3 -- 타입 인자가 생략되어 있어도 타입유추로 알아서 척척
id @Int 3

3

3

In [29]:
id         "hello" -- 타입 인자가 생략되어 있어도 타입유추로 알아서 척척
id @String "hello"

"hello"

"hello"

In [30]:
id @Int "hello" -- 명시적으로 구체화한 타입과 다르게 사용하면 타입 오류

: 

위에서 살펴본 바와 같이 하스켈에서 다형 함수 적용은 System F와 사실상 같은 문법이다.
하스켈은 타입유추를 잘 지원하므로 대부분의 경우 명시적 타입적용을 생략하더라도
컴파일러가 이후에 연이어 적용하는 인자로부터 필요한 타입 정보를 얻어 드러나 보이지 않게 타입적용이 수행된다.

한편, 하스켈에는 타입요약식($\Lambda \alpha.e$)에 직접적으로 대응되는 문법요소는 없다.
다만 아래와 같이 함수 선언에 앞서 `myid :: forall a. a -> a` 또는 줄여서 `myid :: a -> a`로
다형타입을 선언함으로써
`myid`를 System F의 타입요약식으로 선언한 것과 같은 효과를 볼 수 있다.
참고로, 타입 선언 없이도 마찬가지 다형타입이 유추되지만
`@`기호를 사용하는 명시적 타입적용은 사용할 수 없다.

In [31]:
myid :: a -> a
myid = \x -> x

In [32]:
:type myid
:type myid @Int
:type myid @String

myid :: forall a. a -> a

myid @Int :: Int -> Int

myid @String :: String -> String

## 람다계산법과 데이터 타입
타입없는 람다계산법은 튜링기계와 마찬가지로 모든 계산을 표현할 수 있다는 처치--튜링 명제를 언급한 바 있다.
우리가 흔히 접하는 프로그래밍언어에서는 진리값이나 정수와 같은 기본 타입을 비롯해
순서쌍이나 리스트 등의 복합적 데이터 타입을 다루는 계산을 한다.
그런데 함수만 있는 람다계산법으로 어떻게 이런 계산을 표현했을까?
람다계산법의 창시자 알론조 처치가 데이터를 함수로 표현하는 방법으로
고안한
\index{처치 인코딩|see{Church encoding}}\index{Church encoding|see{처치 인코딩}}처치 인코딩(Church encoding)을 통해 가능했던 것이다.

### 진리값

$\begin{array}{rl}
\texttt{true}  ~=& \lambda x.\lambda y.x \\
\texttt{false} ~=& \lambda x.\lambda y.y \\
\texttt{ifte}  ~=& \lambda b.\lambda x.\lambda y.b~x~y ~\equiv_\eta~ \lambda b.b
\end{array}$

이해하기 좋은 기초적인 사례로
진리값 상수에 대한 처치 인코딩(그림 \ref{fig:ChurchBool})을 살펴보자.
프로그래밍언어에서 진리값을 활용하는 대표적인 요소는 바로 `if` $b$ `then` $e_1$ `else` $e_0$와
같은 형태의 조건식(conditional expression)인데, 이를 람다식의 함수 `ifte`로 생각하면
`ifte` $b$ $e_1$ $e_0$에서 $b$가 `true`로 계산될 경우는 $e_1$과 같도록 $b$가 `false`로
계산될 경우는 $e_0$과 같이 동작해야 한다. 처치 인코딩의 핵심 아이디어는 데이터를 사용하는
대표적인 연산의 실제 의미를 데이터 상수 그 자체에 담아놓자는 것이다. 진리값의 경우
조건식의 유의미한 계산은 진리값 상수를 인코딩한 `true`나 `false`에 담겨있다는 말이다.
그래서 조건식을 표현하는 `ifte`는 항등함수의 형태로 사실상 유의미한 계산 없이
진리값 인자를 그대로 내어주도록 (즉, `ifte` $b$ $e_1$ $e_0$ $\equiv_\beta$ $b~e_1~e_0$)
인코딩되어 있다. 이를 우리가 잘 아는 조건식의 동작에 맞추려면,
`true` $e_1$ $e_0$ $\equiv_\beta$ $e_1$이고
`false` $e_1$ $e_0$ $\equiv_\beta$ $e_0$가 되도록,
진리값 상수 `true`를 $\lambda x.\lambda y.x$로
`false`를 $\lambda x.\lambda y.y$로 인코딩하면 된다.

그런데 단순타입 람다계산법(STLC)에서는 처치 인코딩과 같이 데이터 상수를 별도로 설정하지 않고
순수한 람다식만으로 인코딩해 활용하기는 곤란하다.
조건식을 나타내는 `ifte`에 $\texttt{BOOL}\to T\to T$와 같은 타입을 부여하고
그에 맞춰 `true`와 `false`도 $\lambda x{:}T.\lambda y{:}T.x$와 $\lambda x{:}T.\lambda y{:}T.y$로 인코딩해야 하는데,
문제는 $T$를 어떤 타입으로 정하면 다른 타입에 대한 조건식을 작성할 수 없게 된다.
예를 들어 $\texttt{ifte} : \texttt{BOOL}\to \texttt{Int}\to \texttt{Int}$로 한다면
정수를 계산하는 조건식은 작성할 수 있지만 문자열을 계산하는 조건식은 작성할 수 없다.
굳이 억지로 해보자면\newline\indent
$\texttt{ifte}_\texttt{Int} : \texttt{BOOL}_\texttt{Int}\to \texttt{Int}\to \texttt{Int}$ 및 $\texttt{true}_\texttt{Int}$와 $\texttt{false}_\texttt{Int}$,\newline\indent
$\texttt{ifte}_\texttt{String} : \texttt{BOOL}_\texttt{String}\to \texttt{String}\to \texttt{String}$ 및 $\texttt{true}_\texttt{String}$와 $\texttt{false}_\texttt{String}$,\newline\indent
$~\vdots~$\newline
이렇게 계산하려는 각각의 타입 $T$마다
$\texttt{ifte}_T$ 및 $\texttt{true}_T$와 $\texttt{false}_T$로
$\texttt{BOOL}_T$ 타입을 한벌씩 따로 인코딩해야 하므로
불편하고 혼란스러울 따름이다.

다형 람다계산법(System F)에서는 모든 타입에 대해 일관되게 동작을 하는 식에
하나의 다형타입($\forall \alpha.\tau$)을 부여하는 것이 가능하며,
다형타입이 부여된 하나의 식을 필요에 따라 타입적용을 통해 각각의 타입으로 구체화하여 활용할 수 있다.
System F에서와 같은 다형타입으로 표현할 수 있는 이러한 특성을
\index{매개변수 다형성|see{parametric polymorphism}}\index{parametric polymorphism|see{매개변수 다형성}}\`매개변수 다형성'(parametric polymorphism)이라 한다.
매개변수 다형성을 이용하면 처치 인코딩과 마찬가지로 순수한 람다식만으로 System F에서
데이터를 표현하는 것이 가능하며 이를
\index{뵙--베라르두치 인코딩|see{B\"ohm--Berarducci encoding}}\index{B\"ohm-Berarducci encoding|see{뵘--베라르두치 인코딩}}뵘--베라르두치\cite{BoehmBerarducci1985} 인코딩이라 한다.
진리값에 대한 뵘--베라르두치 인코딩은 그림 \ref{fig:BBenc}와 같다.
진리값 상수 `true`와 `false`가 타입요약식($\Lambda \alpha.e$)의 형태로 인코딩되어 있으므로
조건식의 인코딩을 활용할 때 $\texttt{ifte}~b~@\tau~e_1~e_0$처럼 중간에 타입인자($\tau$)에
타입적용을 해야 하며, 이 때 $e_1:\tau$이고 $e_2:\tau$이어야 한다.

\begin{align*}\!\!\!\!\!\!\!\!
\texttt{BOOL}  ~=~& \forall \alpha.\,\alpha\to\alpha\to\alpha \\ \!\!\!\!\!\!\!\!
\texttt{true}  ~=~& \Lambda\alpha.\,\lambda x{:}\alpha.\,\lambda y{:}\alpha.\,x \\ \!\!\!\!\!\!\!\!
\texttt{false} ~=~& \Lambda\alpha.\,\lambda x{:}\alpha.\,\lambda y{:}\alpha.\,y \\ \!\!\!\!\!\!\!\!
\texttt{ifte}  ~=~& \lambda b{:}\texttt{BOOL}.\,b\\
\end{align*}

System F를 기반으로 하는 하스켈로 진리값의 뵘--베라르두치 인코딩을 아래와 같이 옮길 수 있다.
하스켈에서는 타입요약과 타입적용이 직접 드러나지 않으며 람다요약식의 파라미터에 타입 정보를 표시하지 않아도
타입이 유추되므로 System F에서보다 진리값의 인코딩이 조금 더 간결한 형태로 표현된다.

In [33]:
type BOOL = forall a. a -> a -> a 
false :: BOOL
false = \x -> \y -> y
true :: BOOL
true  = \x -> \y -> x
ifte :: BOOL -> BOOL
ifte = \b -> b

\noindent
다음과 같이 여러 타입에 대해 `ifte`를 활용해 조건식을 표현할 수 있다.

In [34]:
ifte true  111     100
ifte false "hello" "world"

111

"world"

\noindent
또한, 이렇게 인코딩된 진리값에 대한 논리곱, 논리합, 논리부정도
다음과 같이 순수한 람다식으로 선언할 수 있다.

In [35]:
andB :: BOOL -> BOOL -> BOOL
andB = \b1 -> \b2 -> b1 b2 b1
orB  :: BOOL -> BOOL -> BOOL
orB  = \b1 -> \b2 -> b1 b1 b2
notB :: BOOL -> BOOL
notB = \b -> \x -> \y -> b y x

In [36]:
true  "true" "false"  -- 진리값이 true인지 false인지
false "true" "false"  -- 이런 방식으로 확인할 수 있다 

"true"

"false"

In [37]:
andB false false "true" "false"
andB false true  "true" "false"
andB true  false "true" "false"
andB true  true  "true" "false"

"false"

"false"

"false"

"true"

In [38]:
orB false false "true" "false"
orB false true  "true" "false"
orB true  false "true" "false"
orB true  true  "true" "false"

"false"

"true"

"true"

"true"

In [39]:
notB false "true" "false"
notB true  "true" "false"

"true"

"false"

두 개의 상수로 이루어진 타입을 이항 다형함수 타입인
$\forall \alpha.\,\alpha\to\alpha\to\alpha$로 인코딩한 것처럼
세 개의 상수로 이루어진 데이터 타입은 삼항 다형함수 타입인
$\forall \alpha.\,\alpha\to\alpha\to\alpha\to\alpha$로 인코딩하며,
일반적으로 $n$개의 상수로 이루어진 데이터 타입을 $n$항 다형함수 타입인
$\forall \alpha.\underbrace{\alpha\to\cdots\to\alpha}_{n-\text{ary}}\to\alpha$로 인코딩할 수 있다.
그러니까 단 한 개의 상수로만 이루어진 유닛(unit) 타입은 $\forall \alpha.\,\alpha\to\alpha$로,
아무 상수도 없는 보이드(void) 타입은 $\forall \alpha.\,\alpha$로 인코딩할 수 있다. 앞서 한번
언급한 바도 있지만 C나 Java와 같은 언어에서 주로 리턴값을 신경쓰지 않는 함수의 리턴 타입으로
쓰이는 `void`는 그 타입의 (정상적인) 값이 없는 것이 아니라 값이 있지만 어차피 하나로 정해져 있으므로
신경쓰지 않겠다는 유닛 타입에 해당하므로 프로그래밍언어 이론의 관점에서는 상당히
혼란스러운 이름이 업계의 주류 프로그래밍언어의 키워드로 사용되는 셈이다. 다행히
최근에 등장한 업계에서 주목받는 언어들 중에는 코틀린(Kotlin)이나 러스트(Rust)등이
유닛 타입이라는 제대로 된 이름으로 부르고 있다.

### 자연수
단순히 상수로만 이루어지지 않고 좀더 복잡한 재귀적 데이터 타입의 뵘--베라르두치 인코딩에 대해서도 알아보자.
하스켈에서 간단한 재귀적 데이터 타입의 사례를 들자면, 자연수(`Nat`)를 다음과 같이
자연수를 받아 하나 더 큰 자연수를 구성하는 데이터 생성자 `S`와 0을 나타내는 데이터 상수 `Z`로 
이루어진 다음과 같은 데이터 타입으로 선언할 수 있다.

In [40]:
data Nat = S Nat | Z   deriving (Show)

In [41]:
:type S
:type Z

S :: Nat -> Nat

Z :: Nat

In [42]:
Z          -- 0
S Z        -- 1
S(S Z)     -- 2
S(S(S Z))  -- 3

Z

S Z

S (S Z)

S (S (S Z))

이렇게 `S`가 몇 번 연달아 적용되었는지가 바로 자연수의 값에 해당되며 `Z`는 마치 맨 끝에 오는 마침표와 같다.
우리에게 가장 익숙한 자연수나 정수의 표현인 십진수는 0부터 9까지 열 개의 기호로 수를 나타낸다.
오늘날의 전자 컴퓨터 내부에서는 0과 1에 해당하는 두 개의 기호로 수를 나타내는 이진수 표현을 사용한다.
위에서 선언한 `Nat`은 따라서 수의 값에 유의미한 영향을 미치는 기호가 `S` 단 한 개이므로 이를 일진수 표현이라고 볼 수 있다.

앞서 진리값에 대한 처치 인코딩이나 뵘--베라르두치 인코딩에서는 진리값을 두 인자 중에서 하나를 선택하는 함수로 인코딩하였다.
자연수에 대한 인코딩의 핵심 아이디어는 자연수를 그 자연수 값만큼 함수적용을 연달아 시키는 고차함수 혹은 그 자연수 값만큼
같은 함수를 반복해 합성한 고차함수로써 인코딩하자는 것이다.
이를테면 3의 처치 인코딩을 $c_3$라고 할 때 $c_3\,f\;x \equiv f(f(f\;x))$라는 이야기다.
그러므로 3의 처치 인코딩 $c_3 = \lambda f.\lambda x.\,f(f(f\;x)))$이며 일반적으로
자연수 $k$의 처치 인코딩 $c_k = \lambda f.\lambda x.\underbrace{f(\ldots(f}_{k\text{번 적용}}\;x)\ldots)$이다.
하스켈 데이터 타입 `Nat`을 이런 방식의 뵘--베라르두치 인코딩으로 옮긴 하스켈 코드는 다음과 같다.

In [43]:
type NAT = forall a. (a -> a) -> a -> a
s :: NAT -> NAT
s = \n   -> \f -> \x -> f (n f x) -- f를 n번 적용한 결과에 f 한 번 더 적용
z :: NAT
z = \f -> \x -> x

\noindent
이렇게 인코딩된 자연수(`NAT`)는 다음처럼 하스켈 정수로 변환해 확인할 수 있다.

In [44]:
z         (+1) 0
s z       (+1) 0
s(s z)    (+1) 0
s(s(s z)) (+1) 0

0

1

2

3

참고로 `(+1)`은 람다식 `(\x -> x+1)`를 줄인 표현이며
마찬가지로 `(1+)`는 람다식 `(\x -> 1+y)`를 줄인 표현이다.
하스켈에서 이렇게 중위 연산식의 어느 한쪽 인자를 빼놓고 괄호로 둘러싼
표현을 잘린식(section)이라 일컬으며 덧셈 말고 다른 중위 연산자로도
`(3*)`이나 `(/2)`와 같이 잘린식을 작성할 수 있다.

### 서로 다른 표현 방식의 자연수로 변환
하스켈의 기본 타입인 `Int`와 앞서 살펴본 `Nat` 및 `NAT` 타입의 자연수를
서로 다른 표현 방식으로 변환하는 함수들을 선언하려 한다.
아래의 `fromInt2Nat` 함수에서 음의 정수는 고려하지 않아도 좋다.
연습문제로 아래 선언을 완성해 보라.

In [45]:
fromInt2Nat :: Int -> Nat
fromInt2Nat 0 = undefined
fromInt2Nat n = undefined

fromNat2Int :: Nat -> Int
fromNat2Int Z     = undefined
fromNat2Int (S n) = undefined

fromNat2NAT :: Nat -> NAT
fromNat2NAT Z     = undefined
fromNat2NAT (S n) = undefined

fromNAT2Nat :: NAT -> Nat
fromNAT2Nat n = undefined

# 하스켈에서 매개변수화된 데이터 타입
이전 장에서 `Maybe a`와 같은 매개변수화된 데이터 타입의 사례를 이미 접해 보았다.
사실 그 전부터 이 책에서는 하스켈의 매개변수화된 데이터 타입이 나타나고 있다.
순서쌍 타입 `(a,b)`는 두 개의 타입변수 `a`와 `b`로 매개변수화된 데이터 타입이며
리스트 타입 `[a]`는 한 개의 타입변수 `a`로 매개변수화된 타입이다.
이렇게 매개변수화된 타입에 대해서는 매개변수를 구체화한 타입에 대한 함수도
선언할 수 있지만 매개변수가 무엇이든 일관되게 동작하는 다형 타입 함수도 선언할 수 있다.
순서쌍과 리스트에 대한 다형 함수의 대표적인 사례로는
이미 표준라이브러리에 선언되어 제공되는 다음과 같은 함수들이 있다.

In [46]:
:type fst
:type snd
:type map
:type filter

fst :: forall a b. (a, b) -> a

snd :: forall a b. (a, b) -> b

map :: forall a b. (a -> b) -> [a] -> [b]

filter :: forall a. (a -> Bool) -> [a] -> [a]

In [47]:
map even [1, 2, 3, 4]        -- a=Int, b=Bool
map show [True, False, True] -- a=Bool, b=String

[False,True,False,True]

["True","False","True"]

In [48]:
import Data.Char (isUpper)
:type isUpper
isUpper 'C' -- 대문자인 경우에 True
isUpper 'c' -- 소문자인 경우는 False

isUpper :: Char -> Bool

True

False

In [49]:
filter even [1, 2, 3, 4, 5, 6]   -- a=Int
filter isUpper ['H','a','B','b'] -- a=Char

[2,4,6]

"HB"

In [50]:
:info String -- 하스켈에서 String은 [Char]의 타입 별명

type String :: *
type String = [Char]
  	-- Defined in ‘GHC.Base’

In [51]:
"AB" == ['A','B'] -- 즉, 문자열이 곧 문자의 리스트다

True

하스켈의 순서쌍은 다음과 같은 데이터 타입으로 선언되어 있다.
참고로, 아래에서 왼쪽의 `(,)`는 타입 생성자이며 오른쪽의 `(,)`는 데이터 생성자이다.
하스켈에서는 타입 계층의 이름공간과 식 계층의 이름공간이 분리되어 있어
똑같은 식별자의 타입 생성자와 데이터 생성자가 같은 프로그램에 있어도 괜찮다.
```haskell

    data (,) a b = (,) a b
    
```
순서쌍의 경우 가독성을 위해 `(a,b)`와 같이 괄호와 쉼표 사에에 쓸 수 있도록 특별한 표기를 허용하는 것일 뿐이다.
그래서 보통의 데이터 타입처럼 전위(prefix) 표기로도 작성이 가능하다.

In [52]:
(,) 1 'a' :: (,) Int Char -- 보통의 데이터 타입과 같은 전위 표현
(1,'a')   :: (Int,Char)   -- 가독성을 위해 특별히 허용하는 표현
(,) 1 'a' == (1,'a')       -- 어떤 방식으로 표현하든 같다

(1,'a')

(1,'a')

True

하스켈의 리스트도 다음과 같은 데이터 타입으로 정의되어 있으며,
순서쌍과 마찬가지로 가독성을 위해 특별한 표기를 허용하는 것일 뿐이다.
```haskell
    data [] a = []              {- 데이터 상수 []로 빈 리스트를 구성 -}
                | (:) a ([] a)  {- 데이터 생성자 (:)로 원소(a)와 리스트([] a)를
                                           묶어 새로운 리스트를 구성 -}
```

In [53]:
(:) 1 ((:) 2 []) :: [] Int -- 보통의 데이터 타입과 같은 전위 표현
1 : 2 : []       :: [Int]  -- 가독성을 위해 특별히 허용하는 표현
[1, 2]           :: [Int]  -- 더 보기좋게 더 특별히 허용하는 표현 
(:) 1 ((:) 2 []) == 1 : 2 : []  -- 어떤 방식으로 표현하든 같다
[1, 2]           == 1 : 2 : []  -- 어떤 방식으로 표현하든 같다

[1,2]

[1,2]

[1,2]

True

True

## 하스켈에서 타입 생성자의 분류
하스켈에서 식은 타입으로 분류된다. 그런데 식의 계층이 아닌 타입의 계층에서도 분류가 필요하다.
`Maybe`, `(,)`, `[]`와 같은 매개변수화된 타입의 식별자는
그 자체로는 타입이 아니며 `Maybe Int`, `(,) Char Bool`, `[] Int` 처럼
적절한 개수의 타입에 적용했을 때 비로소 타입이 되는 타입 생성자(type constructor)이다.
`Int`, `Bool`, `Char`처럼 매개변수화되지 않고 그 자체로 타입인
경우를 0항 타입 생성자(nullary type constructor)로 볼 수 있다.
`Maybe`나 `[]`는 하나의 타입에 적용하면 타입이 되는
1항 타입 생성자(unary type constructor)이며,
`(,)`는 두 개의 타입에 연달아 적용해야 타입이 되는
2항 타입 생성자(binary type constructor)이다.
이러한 타입 생성자의 분류를 카인드(kind)라고 하며, 
다음과 같이 타입 생성자가 어떤 카인드로 분류되는지
하스켈 컴파일러에게 물어볼 수 있다. 

In [54]:
:kind Int  -- 0-ary (nullary) type constructor
:kind Bool -- 0-ary (nullary) type constructor

Int :: *

Bool :: *

In [55]:
:kind Maybe  -- 1-ary (unary) type constructor
:kind []     -- 1-ary (unary) type constructor

Maybe :: * -> *

[] :: * -> *

In [56]:
:kind (,)   -- 2-ary (binary) type constructor
:kind (->)  -- 2-ary (binary) type constructor

(,) :: * -> * -> *

(->) :: * -> * -> *

\noindent
중위 표현의 함수 타입 생성자(`->`)는 매개변수화된 데이터 타입의 식별자는 아니지만
정의역과 공역을 나타내는 두 타입 `t1`과 `t2`에 적용했을 때 비로소 함수 타입 `t1 -> t2`가 되므로,
하스켈에서는 순서쌍 타입 생성자 `(,)`와 같은 카인드(`* -> * -> *`)로 분류된다. 

## 매개변수화된 트리 데이터 타입
데이터 타입을 정의할 때 매개변수 다형성을 활용할 수 없다면
각 노드마다 원소를 포함하는 이진트리(binary tree) 데이터 타입을
다음과 같이 원소의 타입을 달리할 때마다 따로 한벌씩 정의해 주어야 한다.

In [57]:
data BTreeI = BNilI | BNodeI Int  (BTreeI, BTreeI)  deriving Show
data BTreeB = BNilB | BNodeB Bool (BTreeB, BTreeB)  deriving Show
data BTreeC = BNilC | BNodeC Char (BTreeC, BTreeC)  deriving Show

\noindent 위에서 선언한 정수 이진트리(`BTreeI`), 진리값 이진트리(`BTreeB`),
문자 이진트리(`BTreeC`)와 같은 구체적인 원소 타입의 이진트리로 구체화하여
활용할 수 있는 일반적인 이진트리를 매개변수화된 데이터 타입(`BTree a`)으로
다음과 같이\vspace*{-1ex}
```haskell

        data BTree a = BNil | BNode a (BTree a, BTree a)  deriving Show
```
$~$\vspace*{-1ex}\newline\noindent
한번에 정의할 수 있다. 이 때, `a`를 `Int`로 구체화한 `BTree Int`가 `BTreeI`에 해당하고,
`a`를 `Bool`로 구체화한 `BTree Bool`이 `BTreeB`에 해당하며,
`a`를 `Char`로 구체화한 `BTree Char`는 `BTreeC`에 해당한다.

In [58]:
data UTree a = UNil | UNode a (UTree a)                 deriving Show
data BTree a = BNil | BNode a (BTree a,BTree a)         deriving Show
data TTree a = TNil | TNode a (TTree a,TTree a,TTree a) deriving Show

그런데 각 노드마다 몇 갈래의 하위 트리로 갈라지는가에 따라서도
다른 종류의 트리가 된다. 각 노드마다 한 갈래, 두 갈래, 세 갈래로 갈라지는
일진트리(unary tree), 이진트리(binary tree), 삼진트리(ternary tree)를
각각 매개변수화된 데이터 타입 `UTree a`, `BTree a`, `TTree a`로 위와 같이 선언할 수 있다.

\noindent
일진트리, 이진트리, 삼진트리의 사례 각각 하나씩을 위와 같은 그림으로 나타낼 수 있다.
위에서 그림으로 나타낸 각 트리를 앞서 정의한 `UTree`, `BTree`, `TTree` 데이터 타입으로
표현하면 아래와 같다.

In [59]:
UNode 1 (UNode 11 UNil)

BNode 1 ( BNode 11 (BNil,BNil),
          BNode 12 (BNil,BNil) )

TNode 1 ( TNode 11 (TNil,TNil,TNil),
          TNode 12 (TNil,TNil,TNil),
          TNode 13 (TNil,TNil,TNil) )

UNode 1 (UNode 11 UNil)

BNode 1 (BNode 11 (BNil,BNil),BNode 12 (BNil,BNil))

TNode 1 (TNode 11 (TNil,TNil,TNil),TNode 12 (TNil,TNil,TNil),TNode 13 (TNil,TNil,TNil))

위 세 종류의 트리 데이터 타입은 노드에 포함되는 원소의 타입(`a`)을 매개변수화함으로써
일반화되어 있지만, 각 노드에서 몇 갈래로 갈라지는지에 대해서까지는 일반화되어 있지 못하였다.
따라서 각 종류의 트리마다 한벌씩 따로 데이터 타입을 선언하고 있는 것이다.
노드에 포함된 원소의 타입 뿐 아니라 몇 갈래로 갈라지는지에 대해서도 일반화된 트리를
한번에 하스켈 데이터 타입으로 선언할 수는 없을까? 결론적으로 가능하지만,
System F에서 지원하는 다형 타입의 범위를 넘어서는 지금까지 다루지 않았던
새로운 형식의 타입을 동원해야 한다.

## 타입 생성자에 대한 매개변수화를 허용하는 고차 매개변수 다형성
카인드 `*`로 분류되는 타입 뿐 아니라 일반적으로 모든 카인드의 타입 생성자에 대한 매개변수화를
허용하는 더 유연하게 확장된 매개변수 다형성을
\index{고차 매개변수 다형성|see{higher-kinded parametric polymorphism}}\index{higher-kinded parametric polymorphism|see{고차 매개변수 다형성}}\`고차 매개변수 다형성'(higher-kinded parametric polymorphism 혹은
\index{고차 매개변수 다형성|see{higher-order parametric polymorphism}}\index{higher-order parametric polymorphism|see{고차 매개변수 다형성}}higher-order parametric polymorphism)이라 일컫는다.
매개변수 다형성을 다루고 있음이 명확한 문맥에서는 그냥
\`고차 다형성'(higher-kinded polymorphism 혹은 higher-order polymorphism)이라고
줄여서 부른다. 하스켈은 고차 다형성을 지원하므로 앞서 살펴본 원소 타입에 대한
매개변수 `a :: *`에 추가로 각 노드에서 몇 갈래로 갈라지는지를 `f :: * -> *`로
매개변수화한 더 일반적인 트리 데이터 타입인 `GTree f a`를 다음과 같이 선언할 수 있다.
참고로 아래의 데이터 타입 선언에서 `data GTree f a = GNil | GNode a (f (GTree f a))`와
같이 `f`와 `a`의 카인드 표기를 생략하라더도 하스켈 컴파일러가 타입을 유추하듯 카인드도 유추한다.
여기서는 타입 생성자 파라미터 `f`가 타입 파라미터 `a`와는 다른 카인드임을 강조하려고 표기했을 따름이다.

In [60]:
data GTree (f :: * -> *) (a :: *) = GNil | GNode a (f (GTree f a))
{-# LANGUAGE StandaloneDeriving UndecidableInstances #-}
deriving instance (Show a, Show (f (GTree f a))) => Show (GTree f a)

In [61]:
:kind GTree

GTree :: (* -> *) -> * -> *

In [62]:
data UF t = C1 t       deriving Show
data BF t = C2 (t,t)   deriving Show
data TF t = C3 (t,t,t) deriving Show

In [63]:
:kind UF
:kind BF
:kind TF

UF :: * -> *

BF :: * -> *

TF :: * -> *

In [64]:
:kind GTree UF  -- UTree에 해당
:kind GTree BF  -- BTree에 해당
:kind GTree TF  -- TTree에 해당

GTree UF :: * -> *

GTree BF :: * -> *

GTree TF :: * -> *

\begin{align*}
&\text{kind}\quad& 
\kappa\in K ~::=~& {*} \mid \kappa\to\kappa \\
&\text{type constructor}\quad&
\tau\in T ~::= ~& t \mid \tau\to\tau \mid \alpha \mid \forall\alpha{\,:\,}\kappa.\,\tau \mid \lambda\alpha{\,:\,}\kappa.\,\tau \mid \tau~\tau \\
&\text{expression}\quad&
e\in E ~::=~ & c \mid x \mid \lambda x{\,:\,}\tau.\,e \mid e~e \mid \Lambda \alpha{\,:\,}\kappa.\,e \mid e\,@\tau \\[1.5ex]
&\text{카인드 규칙}&\\[-7.5ex]
\end{align*}
$$\qquad\qquad
\Delta ::= t{\,:\,}\kappa,\Delta \mid c{\,:\,}\tau,\Delta \mid \cdot
\qquad
\Gamma ::= \alpha{\,:\,}\kappa,\Gamma \mid x{\,:\,}\tau,\Gamma \mid \cdot 
$$
$$
\frac{~ ~}{~ \Gamma \vdash t:\kappa ~}
({\scriptstyle t{\,:\,}\kappa\;\in\,\Delta})
\quad
\frac{~ \Gamma\vdash\tau_1:* \quad \Gamma\vdash\tau_2:* ~}{~ \Gamma \vdash \tau_1 \to \tau_2 : * ~}
\quad
\frac{~ ~}{~ \Gamma \vdash \alpha:\kappa ~}
({\scriptstyle \alpha{\,:\,}\kappa\;\in\,\Gamma})
$$
$$
\frac{~ \alpha{\,:\,}\kappa,\,\Gamma \vdash \tau : * ~}{~ \Gamma \vdash \forall\alpha{\,:\,}\kappa.\,\tau : * ~}
\quad
\frac{~ \alpha{\,:\,}\kappa_2\,\Gamma\vdash\tau : \kappa ~}{~ \Gamma\vdash\lambda\alpha{\,:\,}\kappa_2.\,\tau : \kappa~}
\quad
\frac{~ \Gamma\vdash\tau_1 : \kappa_2\to\kappa \quad \Gamma\vdash\tau_2 : \kappa_2 ~}{~ \Gamma\vdash\tau_1~\tau_2 : \kappa~}
$$

$~$\newline

\index{람다계산법!타입있는 람다계산법!고차 다형 람다계산법}\index{$\lambda$-calculus!typed $\lambda$-calculus!higher-order polymorphic $\lambda$-calculus}System F의 창시자인 지라르는 이후에 타입 뿐 아니라
타입 생성자에 대한 매개변수를 포함하도록 기존의 System F를 확장한
\index{System F$_\omega$|see{고차 다형 람다계산법}}\index{System F$_\omega$|see{higher-order polymorphic $\lambda$-calculus}}System F$_\omega$라는
\index{고차 다형 람다계산법|see{higher-order polymorphic $\lambda$-calculus}}\index{higher-order polymorphic $\lambda$-calculus|see{고차 다형 람다계산법}}고차 다형 람다계산법을 고안하였다\cite{Girard1972}.
참고로 System F$_\omega$의 문법구조와 카인드 규칙은 그림 \ref{fig:SystemFw}에
나타나 있으며, System F와 대조되는 특징을 정리하면 다음과 같다.\vspace*{-1ex}

- 타입 생성자 변수($\alpha$)가 어떤 카인드($\kappa$)인지 문법구조에 함께 표시한다.
- 타입 계층에 STLC를 갖다놓은 것처럼
  타입 생성자를 타입 생성자에 대응시키는 타입함수($\lambda\alpha{:}\kappa.\tau$)와
  타입 생성자 적용($\tau~\tau$)이 추가되어 있다.
- 올바르게 구성된 타입 생성자인지 여부가 카인드 규칙에 따라 규정된다.

하스켈에서 \`타입 생성자'라고 매번 부르는 것이 번거롭기 때문에 그냥 \`타입'으로 줄여 부르는 경우도 많다.
예를 들어 `Maybe :: * -> *`는 타입 생성자이며 `Maybe Int :: *`는 타입이지만,
그냥 짧게 `Maybe` 타입 이라고 부르곤 한다.

\section*{요점정리}
* 처치가 고안한 (타입없는) 람다계산법으로
  튜링이 고안한 튜링기계과 똑같은 범위의 계산을 표현할 수 있다. 지금까지도 바로 이 범위를
  기계적으로 수행할 수 있는 (즉, 컴퓨터로 돌릴 수 있는)
  \`계산'의 이론적 한계 범위로 받아들이며 이를 처치--튜링 명제(Church--Turing thesis)라고 일컫는다.
* 타입없는 람다계산법의 문법구조는 이름, 타입요약식, 타입적용식 이렇게 단 세 종류의 요소로 구성됨에도 불구하고
  끝나지 않는 계산을 포함한 일반적인 재귀를 표현할 수 있는 고정점(fixpoint) 조합자를 람다식으로 작성 가능하다.
* 고정점 조합자가 주어지면 직접 재귀함수를 선언하지 않더라도 고정점 조합자를 재귀함수가 아닌 함수에 적용함으로써 원하는 재귀함수를 얼마든지 작성할 수 있다.
* 타입없는 람다계산법에서는 처치 인코딩으로 진리값이나 자연수 등의 데이터를 함수로서 인코딩해 순수한 람다식만으로 이러한 데이터 및 그에 대한 연산을 표현할 수 있다.
* 지라르와 레이놀즈는 단순타입 람다계산법에서 다루는 타입에 타입변수($\alpha$)와 다형타입($\forall\alpha.\tau$)을 추가한 System F라고도 불리는 다형 람다계산법을 독립적으로 고안하였다.
* System F의 람다식에는 STLC의 람다식에 타입요약식($\Lambda\alpha.e$)와 타입적용식($e\,@\tau$)가 추가되어 있다.
* 다형타입을 지원하는 하스켈 언어의 핵심에는 System F가 있다고 볼 수 있지만 타입요약식과 타입적용식이 일반적으로 겉으로 드러나 보이지 않으며 타입유추 과정에서 하스켈 컴파일러가 보이지 않게 처리한다고 이해할 수 있다.
* System F에서는 타입없는 람다계산법의 처치 인코딩과 마찬가지 방식으로 데이터 타입을 인코딩할 수 있으며 이를 뵘--베라두치 인코딩이라 한다.
* 지라르는 System F에서 타입변수가 타입만을 대표하는 것을 일반화하여 타입 생성자를 대표할 수 있도록 확장한 System F$_\omega$라고도 불리는 고차 다형 람다계산법을 고안하였다.
  하스켈도 사실 보통의 매개변수 다형성만이 아닌 고차 다형성까지 지원하므로 System F$_\omega$가 하스켈 언어의 핵심에 있다고 이해할 수 있다.
* 식(expression)이 타입(type)으로 분류되는 것처럼 System F$_\omega$에서 타입 생성자는 카인드(kind)로 분류된다.
  타입 규칙으로 식이 타입에 맞게 구성되었는지 규정되듯 카인드 규칙으로 올바르게 구성된 타입 생성자인지 규정된다.
* 하스켈에서 파라미터화된 데이터 타입의 식별자는 그 자체로는 타입이 아니며 적절한 타입 인자에 적용하면 타입이 되는 타입 생성자다.
  예컨대, 타입 생성자 `Maybe :: * -> *`를 타입 `T :: *`에 적용하면 `Maybe T :: *`라는 타입이 된다.

\section*{연습문제}
1. 피보나치 수를 계산하는 함수 `fib :: Integer -> Integer`를 직접 재귀함수로 선언해 보고 또  고정점 고차함수 `fix`와 재귀적이지 않은 함수를 통해서도 선언해 보라. 
1. `[45]`번 셀의 `fromInt2Nat`, `fromNat2Int`, `fromNat2NAT`, `fromNAT2Nat` 함수 선언을 완성하라.
1. `[59]`번 셀에서 각각 다른 데이터 타입으로 표현된 세 그루의 트리를 더 일반적인 `GTree` 데이터 타입으로 모두 옮겨 보라.

\section*{탐구과제}
1. `[62]`번 셀의 `UF`, `BF`, `TF`는 `data`로 선언하는 것보다는 `newtype`으로 선언하는 것이 더 바람직하다.
   하스켈에서 `newtype`은 어떨 때 활용하는지 알아보라.
1. 하스켈 이외에 고차 다형 타입(higher-kinded type 혹은 higher-order type)을 지원하는 언어로는 어떤 것이 있으며
   어떤 방식으로 지원하는지 조사해 보라.
1. STLC에 처치 방식과 커리 방식이 있듯 System F나 System F$_\omega$에도 처치 방식과 커리 방식이 있다.
   책에 소개된 것은 처치 방식이다. 커리 방식의 System F나 System F$_\omega$는 어떤 차이점이 있는지 알아보라.